<a href="https://colab.research.google.com/github/yfrund/ML_practice/blob/main/Tutorial_9_practical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Note: These are simplified examples. In real-world scenarios you'd rely on more data to make judgements (e.g., training and validation loss, test loss, etc.). These are just simple practice tasks.

# Task 1

You're given a deep network with multiple hidden layers.

We train it on some data.

Observe gradient magnitude during backpropagation. Do you see any potential issues?


In [ ]:
!pip install torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# we generate a synthetic dataset
X = torch.randn(100, 10) # 100 samples, 10 features
y = torch.randint(0, 2, (100,)).float()  # some binary targets

# define a simple network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.hidden1 = nn.Linear(10, 20)
        self.hidden2 = nn.Linear(20, 20)
        self.hidden3 = nn.Linear(20, 20)
        self.hidden4 = nn.Linear(20, 20)
        self.hidden5 = nn.Linear(20, 20)
        self.output = nn.Linear(20, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.hidden1(x))
        x = self.activation(self.hidden2(x))
        x = self.activation(self.hidden3(x))
        x = self.activation(self.hidden4(x))
        x = self.activation(self.hidden5(x))
        return self.activation(self.output(x))

# initializations
model = SimpleNet()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# training
for epoch in range(10):
    print(f'\n\nEpoch #{epoch}.\n\n')
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.squeeze(), y) #squeezing to ensure the same size between input and output
    print(f'Loss: {loss.item()}.\n')
    loss.backward()

    # gradient magnitudes - norm computes the magnitude of vectors for example purposes
    print("Gradient magnitudes for each layer:\n")
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"{name}: {param.grad.norm().item()}\n") # norm to compute vector magnitude

    optimizer.step()


How can you solve the issue?

#Task 2.

You're training another simple model.

Observe the output. What's going on?

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


torch.manual_seed(42)
X = torch.randn(1000, 10)  # 1000 samples, 10 features - a small dataset
y = torch.randint(0, 2, (1000,)).float()  # binary targets

# split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# convert to PyTorch tensors
X_train, X_val, X_test = map(torch.tensor, (X_train, X_val, X_test))
y_train, y_val, y_test = map(torch.tensor, (y_train, y_val, y_test))

# define a simple network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.hidden1 = nn.Linear(10, 100)
        self.hidden2 = nn.Linear(100, 100) # lots of neurons
        self.hidden3 = nn.Linear(100, 100)
        self.output = nn.Linear(100, 1)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.hidden1(x))
        x = self.activation(self.hidden2(x))
        x = self.activation(self.hidden3(x))
        return self.output(x)

# initialisations
model = SimpleNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training
epochs = 50
batch_size = 64

# a simple function to calculate accuracies (this is a simple binary classificator - that's why we'll use accuracy as an example; perplexity would be more appropriate for language generation)
def calculate_accuracy(outputs, targets):
    predictions = torch.sigmoid(outputs).round()
    return (predictions == targets).float().mean().item()

for epoch in range(epochs):

    model.train() # now the model is in training mode
    for i in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        outputs = model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # switch to evaluation mode
    model.eval()
    with torch.no_grad():
        train_outputs = model(X_train).squeeze()
        val_outputs = model(X_val).squeeze()
        test_outputs = model(X_test).squeeze()

        train_acc = calculate_accuracy(train_outputs, y_train)
        val_acc = calculate_accuracy(val_outputs, y_val)


        print(f"\n\nEpoch {epoch}:")
        print(f"  Training Accuracy: {train_acc:.4f}")
        print(f"  Validation Accuracy: {val_acc:.4f}")

model.eval()
with torch.no_grad():
    test_outputs = model(X_test).squeeze()
    test_acc = calculate_accuracy(test_outputs, y_test)

# print final accuracies
print("\nFinal Accuracies:")
print(f"Training Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")


How can you solve the issue?